<a href="https://colab.research.google.com/github/semhoun/omnius/blob/main/nb/FrenchSFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>


# Model List
[Model lists](https://huggingface.co/unsloth):
 - unsloth/Meta-Llama-3.1-8B-bnb-4bit ***Llama-3.1 15 trillion tokens model 2x faster!***
 - unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit
 - unsloth/Meta-Llama-3.1-70B-bnb-4bit
 - unsloth/Meta-Llama-3.1-405B-bnb-4bit ***We also uploaded 4bit for 405b!***
 - unsloth/Mistral-Nemo-Base-2407-bnb-4bit ***New Mistral 12b 2x faster!***
 - unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit
 - unsloth/mistral-7b-v0.3-bnb-4bit ***Mistral v3 2x faster!***
 - unsloth/mistral-7b-instruct-v0.3-bnb-4bit
 - unsloth/Phi-3.5-mini-instruct ***Phi-3.5 2x faster!***
 - unsloth/Phi-3-medium-4k-instruct
 - unsloth/gemma-2-9b-bnb-4bit
 - unsloth/gemma-2-27b-bnb-4bit ***Gemma 2x faster!***
 - unsloth/SmolLM3-3B-bnb-4bit

Remember to go to https://huggingface.co/settings/tokens for a token!


# Definitions


In [ ]:
import os

if "COLAB_" in "".join(os.environ.keys()):
  from google.colab import userdata
  hf_token = userdata.get('HuggingFaceToken')
else:
  hf_token = os.environ["HUGGINGFACETOKEN"]

#source_model = "mistralai/Mistral-Nemo-Instruct-2407"
#source_model = "mistralai/Mistral-7B-Instruct-v0.3"
#source_model = "nvidia/NVIDIA-Nemotron-Nano-9B-v2"
source_model = "HuggingFaceTB/SmolLM3-3B"

quantization_method = ["q8_0", "f16"]
#quantization_method = ["q8_0", "f16", "q4_k_m", , "q4_0"]
model_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
per_device_train_batch_size = 1

#model_name = "Claire-12B-v1.0"
model_name = "Claire-Dev"
hf_username = "nsemhoun"

debug = True
save_local = False
save_hf = True

# Installation

In [ ]:
%%capture
import os, re
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# We're installing the latest Torch, Triton, OpenAI's Triton kernels, Transformers and Unsloth!
!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers>=4.55.3" \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

# Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = source_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = model_4bit,
    token = hf_token,
)

#Dataset Preparation and Processing
We now use the `ChatML` format for conversation style finetunes. ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Normally one has to train `<|im_start|>` and `<|im_end|>`. We instead map `<|im_end|>` to be the EOS token, and leave `<|im_start|>` as is. This requires no additional training of additional tokens.

More info on chat templates on [our wiki page!](https://github.com/unslothai/unsloth/wiki#chat-templates)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style if needed
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    texts = []
    if (examples.get('system') == None):
      transformed_data = [
          {"role": "user", "content": examples.get('question')},
          {"role": "assistant", "content": examples.get('chosen')},
      ]
    else:
      transformed_data = [
          {"role": "system", "content": examples.get('system')},
          {"role": "user", "content": examples.get('question')},
          {"role": "assistant", "content": examples.get('chosen')},
      ]
    text = tokenizer.apply_chat_template(transformed_data, tokenize = False, add_generation_prompt = False)
    return { "text" : text, }
pass

from datasets import load_dataset
dataset = load_dataset("jpacifico/french-orca-dpo-pairs-revised", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = False,)

Let's see how the format works by printing the 5th element

In [ ]:
import pprint
row = dataset[5]
print('SYSTEM: ' + '=' * 50)
pprint.pprint(row["system"])
print('INSTRUCTION: ' + '=' * 50)
pprint.pprint(row["question"])
print('ACCEPTED: ' + '=' * 50)
pprint.pprint(row["chosen"])
print('CHAT TEMPLATE: ' + '=' * 50)
pprint.pprint(row["text"])


# Model Training Setup


Add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
lora_rank = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Setup the SFT traincer with approprite arguments.

*We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.*

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, #1
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        max_steps = 60 if debug else -1, # None for full run
        num_train_epochs = 0 if debug else 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # remove to activate WandDB
    ),
)

# Training Execution
Execute the training process with the configured trainer and monitor the training progress.

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# Model Merge

In [ ]:
#model = model.merge_and_unload()

<a name="Inference"></a>
# Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

# Saving

<a name="Save"></a>
# Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
messages = [
    {"role": "user", "content": "Quel est la fameuse grande tour à Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

## Saving to float16 for VLLM (safetensors)

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
if model_4bit:
  # Merge to 4bit
  if save_local: model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_4bit",)
  if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, tokenizer, save_method = "merged_4bit", token = hf_token)
else:
  # Merge to 16bit
  if save_local: model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_16bit",)
  if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, tokenizer, save_method = "merged_16bit", token = hf_token)

if False:
  # Just LoRA adapters
  if save_local:
    model.save_pretrained(model_name, save_method = "lora",)
    tokenizer.save_pretrained(model_name, save_method = "lora",)
  if save_hf:
    model.push_to_hub(hf_username + "/" + model_name, save_method = "lora", token = hf_token)
    tokenizer.push_to_hub(hf_username + "/" + model_name, save_method = "lora", token = hf_token)


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to local
if save_local:
  model.save_pretrained_gguf(model_name, tokenizer, quantization_method = quantization_method)

if save_hf:
  model.push_to_hub_gguf(hf_username + "/" + model_name, tokenizer, quantization_method = quantization_method, token = hf_token)

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
